### Notebook for finetuning model

Model to finetune is allegro/herbert-base-cased. Do finetunningu zostaną zamrożone wszystkie warstwy oprócz ostatnich 4.

In [35]:
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from torch import softmax
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModel, EvalPrediction, Trainer, TrainingArguments

In [26]:
class HateClassifier(nn.Module):
    def __init__(self, model_name="allegro/herbert-base-cased", num_labels=2):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        hidden_size = self.bert.config.hidden_size

        self.classifier = nn.Linear(hidden_size, num_labels)
        self.dropout = nn.Dropout(0.2)

        for param in self.bert.parameters():
            param.requires_grad = False

        for layer_idx in range(8, 12):
            for param in self.bert.encoder.layer[layer_idx].parameters():
                param.requires_grad = True

    def forward(self, input_ids, attention_mask):
        out = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        cls = out.last_hidden_state[:, 0, :]
        x = self.dropout(cls)
        logits = self.classifier(x)
        return logits


### Prepare Data

In [6]:
df = pd.read_csv("data/train.csv")
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42, stratify=df["Class"])

In [27]:
class HateDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])

        encoded = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_len,
            return_tensors="pt"
        )

        return {
            "input_ids": encoded["input_ids"].squeeze(0),
            "attention_mask": encoded["attention_mask"].squeeze(0),
            "labels": torch.tensor(self.labels[idx], dtype=torch.long)
        }

In [29]:
tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")

train_dataset = HateDataset(train_df["Text"].tolist(), train_df["Class"].tolist(), tokenizer)
val_dataset = HateDataset(val_df["Text"].tolist(), val_df["Class"].tolist(), tokenizer)

### Training

In [23]:
def compute_metrics(eval_pred: EvalPrediction) -> dict:
    y_true = eval_pred.label_ids.ravel()
    logits = torch.from_numpy(eval_pred.predictions)
    y_pred_proba = softmax(logits, dim=1)[:, 1].numpy()

    y_pred = (y_pred_proba >= 0.5).astype(int)

    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    return {
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1
    }

In [39]:
def train(
        model,
        train_dataset,
        val_dataset,
        output_dir="model_output",
        epochs=3,
        batch_size=16,
        learning_rate=2e-5,
        weight_decay=0.01,
        logging_steps=50,
        save_steps=50,
        seed=42
):
    torch.manual_seed(seed)

    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        evaluation_strategy="steps",
        logging_steps=logging_steps,
        save_steps=save_steps,
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        greater_is_better=True,
        seed=seed,
        fp16=torch.cuda.is_available()
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
        disable_tqdm=True
    )

    trainer.train()

    metrics = trainer.evaluate()
    print("Final evaluation metrics:", metrics)
    return trainer, trainer.model

In [40]:
def plot_training(trainer):
    df = pd.DataFrame(trainer.state.log_history)
    df_train = df.dropna(subset=["loss"])
    df_eval = df.dropna(subset=["eval_loss"])

    plt.figure(figsize=(10, 5))

    plt.plot(df_train["step"], df_train["loss"], label="Training Loss")
    plt.plot(df_eval["step"], df_eval["eval_loss"], label="Validation Loss")

    plt.xlabel("Step")
    plt.ylabel("Loss")
    plt.title("Training vs Validation Loss")
    plt.grid(True)
    plt.legend()
    plt.show()


def plot_metrics(trainer):
    df = pd.DataFrame(trainer.state.log_history)
    metrics = ["eval_accuracy", "eval_precision", "eval_recall", "eval_f1"]

    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    axes = axes.flatten()

    for i, metric in enumerate(metrics):
        df_metric = df.dropna(subset=[metric])
        axes[i].plot(df_metric["step"], df_metric[metric], marker='o')
        axes[i].set_title(metric)
        axes[i].set_xlabel("Step")
        axes[i].set_ylabel(metric)
        axes[i].grid(True)

    plt.tight_layout()
    plt.show()


### Try training

In [41]:
device = torch.device("cpu")
model = HateClassifier()
model.to(device)

trainer, trained_model = train(
    model=model,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    output_dir="./hate_model",
    epochs=3,
    batch_size=16,
    learning_rate=2e-5,
    log_gging_steps=1,
)

KeyboardInterrupt: 